In [1]:
import arcpy

import sys

arcpy.env.overwriteOutput = True

In [2]:
default_workspace = r"C:\Users\emykl\OneDrive\Documents\GEOG-5055\Final Project\Default.gdb"

# Check if input default_workspace is valid
if arcpy.Exists(default_workspace): 
    arcpy.env.workspace = default_workspace # Set workspace to workspace
    
else: # Print statement if input workspace is invalid
    print(f"Workspace {default_workspace} does not exist.")

In [3]:
# Set input locations for required files
in_places = r"C:\Users\emykl\OneDrive\Documents\GEOG-5055\Final Project\Default.gdb\Iowa Places"
in_watershed = r"C:\Users\emykl\Downloads\HUC8\HUC8.shp"
iowa_border = r"C:\Users\emykl\OneDrive\Documents\GEOG-5540\iowa_border\iowa_border.shp"
iowa_ngs = r"C:\Users\emykl\OneDrive\Documents\GEOG-5055\Final Project\Default.gdb\Iowa_NGS"
impaired_lakes =  r"C:\Users\emykl\OneDrive\Documents\GEOG-5055\Final Project\Default.gdb\Impaired_Lakes"
impaired_streams = r"C:\Users\emykl\OneDrive\Documents\GEOG-5055\Final Project\Default.gdb\Impaired_Streams"

# Create a list of all data sources that are combined to make a dataset for contaminated water
contam_files = [iowa_ngs, impaired_lakes, impaired_streams]

# Pre-name important ouput files
# Contaiminated water layer
contam_water = "Contam_water"
# Output buffer layer
buffer_merged = "Buffer_Merged"

In [ ]:
# Use select layer by attribute tool to select only those locations from NGS dataset that have Phosphorus
# levels greater than or equal to 0.1 parts per million
arcpy.management.SelectLayerByAttribute(iowa_ngs, "NEW_SELECTION", "P_ICP40 >= 0.1", None)

# Use feature class to geodatabase tool to put the previously set contaminated files features into a new
# geodatabase
arcpy.conversion.FeatureClassToGeodatabase(contam_files, contam_water)

# Set the workspace to this new geodatabase for use in the subsequent for loop
arcpy.env.workspace = contam_water

# list feature classes in this geodatabase
# run for loop going through each feature class to create a 500 meter buffer around each feature class
# and then also rename the feature class
fcs = arcpy.ListFeatureClasses()
for fc in fcs:
    print(f"working on{fc}")

    arcpy.analysis.Buffer(fc, f"Buffer_{fc}", "500 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

# Merge these new buffer feature classes into a combined feature class
arcpy.management.Merge("Buffer_Impaired_Streams;Buffer_Impaired_Lakes;Buffer_Iowa_NGS", buffer_merged)

In [ ]:
# Reset workspace to default to complete spatial processing steps
arcpy.env.workspace = default_workspace

# Use summarize within tool between buffer merged feature class and populations center data to create 
# new output along with statistics table that stores new calculated variable
arcpy.analysis.SummarizeWithin(in_places, buffer_merged, "Places_Buffer", "ONLY_INTERSECTING",
                               "Shape_Area Sum","ADD_SHAPE_SUM", "SQUAREKILOMETERS",
                               None, "NO_MIN_MAJ", "NO_PERCENT", None)

# Use intersect tool bewteen buffer merged feature class and population center data to create new output 
arcpy.analysis.Intersect([buffer_merged, in_places], "Places_Buffer_Int", "ALL", None, "INPUT")

# Use summarize within tool between buffer merged feature class and HUC8 watershed data to create 
# new output along with statistics table that stores new calculated variable
arcpy.analysis.SummarizeWithin(in_watershed, buffer_merged, "HUC8SW", "ONLY_INTERSECTING", "Shape_Area Sum",
                               "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

# Use clip tool to cut off section of the HUC8 shapefile that fall outside of the Iowa border
arcpy.analysis.Clip("HUC8SW", iowa_border, "HUC8SW_Clip", None)